In [2]:
# Copyright (c) Roman Lutz. All rights reserved.
# The use and distribution terms for this software are covered by the
# Eclipse Public License 1.0 (http://opensource.org/licenses/eclipse-1.0.php)
# which can be found in the file LICENSE.md at the root of this distribution.
# By using this software in any fashion, you are agreeing to be bound by
# the terms of this license.
# You must not remove this notice, or any other, from this software.

import pdb
import numpy as np
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import feature_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error
from create_datasets import test_players
import time
from metrics import mean_relative_error
from plots import histogram
import sklearn.model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import sklearn









kernels = ['rbf', 'linear', 'sigmoid']
degrees = [2, 3]
gamma_values = [0.05*k for k in range(1,4)]
C_values = [0.25*k for k in range(1, 5)]
epsilon_values = [0.05*k for k in range(1, 6)]
score = 'neg_mean_squared_error'




def hyperparameter(x, y):

    # elements of this list are dictionaries that contain the best parameters for each kernel to use on test data (2017 data)
    overall_best_params = []

    # linear
    linear_kernel = ['linear']
    linear_parameters = {'kernel':(linear_kernel), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    # Since we are predicting scores and using regression, use SVR()
    svr = sklearn.svm.SVR()
    clf = sklearn.model_selection.GridSearchCV(svr, linear_parameters, verbose=5, scoring=score)
    clf.fit(x, y)
    cvres = clf.cv_results_
    linear_kernel_data = open('linear_kernel_data.txt', 'w')
    print("LINEAR:")
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
        linear_kernel_data.write(str(mean_score) + " " + str(params))
    print("Best: " + str(clf.best_params_))
    overall_best_params.append(clf.best_params_)
    linear_kernel_data.close()

    # rbf
    rbf_kernel = ['rbf']
    rbf_parameters = {'kernel':(rbf_kernel), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    svr = sklearn.svm.SVR()
    clf = sklearn.model_selection.GridSearchCV(svr, rbf_parameters, verbose=5, scoring=score)
    clf.fit(x, y)
    cvres = clf.cv_results_
    rbf_kernel_data = open('rbf_kernel_data.txt', 'w')
    print("RBF:")
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
        rbf_kernel_data.write(str(mean_score) + " " + str(params))
    print("Best: " + str(clf.best_params_))
    overall_best_params.append(clf.best_params_)
    rbf_kernel_data.close()

    # sigmoid
    sigmoid_kernel = ['sigmoid']
    sigmoid_parameters = {'kernel':(sigmoid_kernel), 'C':(C_values), 'gamma':(gamma_values), 'epsilon':(epsilon_values)}
    svr = sklearn.svm.SVR()
    clf = sklearn.model_selection.GridSearchCV(svr, sigmoid_parameters, verbose=5, scoring=score)
    clf.fit(x, y)
    cvres = clf.cv_results_
    sigmoid_kernel_data = open('sigmoid_kernel_data.txt', 'w')
    print("SIGMOID:")
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
        sigmoid_kernel_data.write(str(mean_score) + " " + str(params))
    print("Best: " + str(clf.best_params_))
    overall_best_params.append(clf.best_params_)
    sigmoid_kernel_data.close()

    # overall best parameters to use on test data (2017 season)
    print("RUN BEST PARAMETERS ON TEST DATA (2017):")
    svr = sklearn.svm.SVR()
    clf = sklearn.model_selection.GridSearchCV(svr, (param for param in overall_best_params), verbose=5, scoring=score)
    clf.fit(test_x, test_y)
    cvres = clf.cv_results_
    best_kernel_data = open('best_kernel_data.txt', 'w')
    for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(mean_score, params)
        best_kernel_data.write(str(mean_score) + " " + str(params))
    print("Best: " + str(clf.best_params_))
    best_kernel_data.close()
    print("FEATURE SELECTION FINISHED")







# Only one of the feature selection methods can be chosen
FEATURE_SELECTION = False
MANUAL_FEATURE_SELECTION = False
FEATURE_NORMALIZATION = False
HYPERPARAMETER_SELECTION = True
HISTOGRAM = True


"""
train = np.load('train.npy')
test = np.load('test.npy')

train_x = train[:, 2:36].astype(np.float)
train_y = train[:, 36].astype(np.float)
test_x = test[:, 2:36].astype(np.float)
test_y = test[:, 36].astype(np.float)
pdb.set_trace()
"""
"""
# load data
# indices are
# 0: QB id
# 1: QB name
# 2: QB age
# 3: QB years pro
# 4
# 5-16: last game QB stats
# 17-28: last 10 games QB stats
# 29-32: last game defense stats
# 33-36: last 10 games defense stats
# 37: actual fantasy score = target
"""

train = np.load('train.npy')
test = np.load('test.npy')

train_x = train[:, 2:37].astype(np.float)
train_y = train[:, 37].astype(np.float)
test_x = test[:, 2:37].astype(np.float)
test_y = test[:, 37].astype(np.float)



#kernels = ['rbf', 'linear', 'sigmoid', 'poly']
kernels = ['rbf', 'linear', 'sigmoid']
degrees = [2, 3]
gamma_values = [0.05*k for k in range(1,4)]
C_values = [0.25*k for k in range(1, 5)]
epsilon_values = [0.05*k for k in range(1, 6)]

# Feature Normalization
if FEATURE_NORMALIZATION:
    print 'started feature normalization', time.time()
    x = np.concatenate((train_x, test_x), axis=0)
    x = preprocessing.scale(x)
    train_x = x[:len(train_x)]
    test_x = x[len(train_x):]


# Recursive Feature Elimination with cross-validation (RFECV)
if FEATURE_SELECTION:
    print 'started feature selection', time.time()
    selector = feature_selection.RFECV(estimator=SVR(kernel='linear'), step=3, cv=5)
    selector.fit(train_x, train_y)
    train_x = selector.transform(train_x)
    test_x = selector.transform(test_x)
    print selector.ranking_
elif MANUAL_FEATURE_SELECTION: # leave out the two point attempts
    manual_indices = [0, 1, 2, 3, 4, 5, 8, 9, 10, 13, 14, 15, 16, 17, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33]
    train_x = train_x[:, manual_indices]
    test_x = test_x[:, manual_indices]

# USE NORMAL KFOLD
# hyperparameter selection
if HYPERPARAMETER_SELECTION:
    hyperparameter(train_x, train_y)


Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................
[CV]  epsilon=0.05, C=0.25, gamma=0.05, kernel=linear, score=-55.3135948629, total=  32.2s
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.4s remaining:    0.0s


[CV]  epsilon=0.05, C=0.25, gamma=0.05, kernel=linear, score=-58.9514443037, total=  24.3s
[CV] epsilon=0.05, C=0.25, gamma=0.05, kernel=linear .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   57.0s remaining:    0.0s


[CV]  epsilon=0.05, C=0.25, gamma=0.05, kernel=linear, score=-56.9030479034, total=  31.3s
[CV] epsilon=0.05, C=0.25, gamma=0.1, kernel=linear ..................


...

[CV]  epsilon=0.1, C=0.75, gamma=0.15, kernel=sigmoid, score=-68.1021173913, total=   0.5s
[CV] epsilon=0.1, C=0.75, gamma=0.15, kernel=sigmoid .................
[CV]  epsilon=0.1, C=0.75, gamma=0.15, kernel=sigmoid, score=-66.7920499605, total=   0.5s
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid ................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid, score=-67.7106530435, total=   0.5s
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid ................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid, score=-68.1021173913, total=   0.5s
[CV] epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid ................
[CV]  epsilon=0.15, C=0.75, gamma=0.05, kernel=sigmoid, score=-66.7713347826, total=   0.5s
[CV] epsilon=0.15, C=0.75, gamma=0.1, kernel=sigmoid .................
[CV]  epsilon=0.15, C=0.75, gamma=0.1, kernel=sigmoid, score=-67.7106530435, total=   0.5s
[CV] epsilon=0.15, C=0.75, gamma=0.1, kernel=sigmoid .................
[CV]  epsilon=0.15, C=0.7

[CV]  epsilon=0.15, C=1.0, gamma=0.1, kernel=sigmoid, score=-68.1021173913, total=   0.5s
[CV] epsilon=0.15, C=1.0, gamma=0.1, kernel=sigmoid ..................
[CV]  epsilon=0.15, C=1.0, gamma=0.1, kernel=sigmoid, score=-66.7713347826, total=   0.5s
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid .................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid, score=-67.7106530435, total=   0.5s
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid .................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid, score=-68.1021173913, total=   0.5s
[CV] epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid .................
[CV]  epsilon=0.15, C=1.0, gamma=0.15, kernel=sigmoid, score=-66.7713347826, total=   0.5s
[CV] epsilon=0.2, C=1.0, gamma=0.05, kernel=sigmoid ..................
[CV]  epsilon=0.2, C=1.0, gamma=0.05, kernel=sigmoid, score=-67.7315433992, total=   0.5s
[CV] epsilon=0.2, C=1.0, gamma=0.05, kernel=sigmoid ..................
[CV]  epsilon=0.2, C=1.0, gamma

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:  2.1min finished


SIGMOID:
(-67.54284339920949, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.05, 'kernel': 'sigmoid'})
(-67.54284339920949, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.1, 'kernel': 'sigmoid'})
(-67.54284339920949, {'epsilon': 0.05, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'sigmoid'})
(-67.52804334650858, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.05, 'kernel': 'sigmoid'})
(-67.52804334650858, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.1, 'kernel': 'sigmoid'})
(-67.52804334650858, {'epsilon': 0.1, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'sigmoid'})
(-67.52803507246377, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.05, 'kernel': 'sigmoid'})
(-67.52803507246377, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.1, 'kernel': 'sigmoid'})
(-67.52803507246377, {'epsilon': 0.15000000000000002, 'C': 0.25, 'gamma': 0.15000000000000002, 'kernel': 'sigmoid'})
(-67.53543096179183, {'epsilon': 0.2, 'C': 0.25, 'gamma': 0.05, 'kernel': 'sigmoid'})
(-67.53543096179183, {'epsilon': 0.2, 

ValueError: Parameter values for parameter (epsilon) need to be a sequence(but not a string) or np.ndarray.